<a href="https://colab.research.google.com/github/MohammadYehya/SegmentAnythingModel_SAM/blob/main/SAM3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-knjz4gno
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-knjz4gno
  Resolved https://github.com/huggingface/transformers.git to commit cac0a28c83cf87b7a05495de3177099c635ba852
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.1/516.1 kB 32.4 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-5.0.0.dev0-py3-none-any.whl size=10801169 sha256=a873bb25fcdbf6f5ec788c046c4bf6e1de33cb7888d86506da67e7559aa81ecb
  Stored in directory: /tmp/pip-ephem-wheel-cache-whre7s30/wheels/54/cb/3f/83103de5575c534436d6a4686686dead458238dfaf1147e98d
Successfully built transformers
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstalling huggingface-h

In [2]:
from transformers import Sam3Processor, Sam3Model
import torch
from PIL import Image
import requests
from google.colab import files
import cv2
import numpy as np

In [3]:
# Upload image (Colab) or load locally
uploaded = files.upload()
img_path = list(uploaded.keys())[0]
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
print("Loaded image", img_path, "shape", img.shape)

Saving test2.jpg to test2.jpg
Loaded image test2.jpg shape (300, 450, 3)


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = Sam3Model.from_pretrained("facebook/sam3").to(device)
processor = Sam3Processor.from_pretrained("facebook/sam3")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/facebook/sam3.
401 Client Error. (Request ID: Root=1-692c8387-3ead14ca198fd8865c4bd476;fd63e17a-c186-4928-ab0e-a46587b2dd60)

Cannot access gated repo for url https://huggingface.co/facebook/sam3/resolve/main/config.json.
Access to model facebook/sam3 is restricted. You must have access to it and be authenticated to access it. Please log in.

In [ ]:

def overlay_mask_translucent(img, mask, color=(0,255,0), alpha=0.4):
    overlay = img.copy()
    mask_bool = mask.numpy().astype(bool)
    overlay[mask_bool] = np.array(color, dtype=np.uint8)
    blended = img.copy()
    blended[mask_bool] = (img[mask_bool] * (1 - alpha) + overlay[mask_bool] * alpha).astype(np.uint8)
    return blended

In [ ]:
# Segment using text prompt
inputs = processor(images=img, text="cat", return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model(**inputs)

# Post-process results
results = processor.post_process_instance_segmentation(
    outputs,
    threshold=0.5,
    mask_threshold=0.5,
    target_sizes=inputs.get("original_sizes").tolist()
)[0]

print(f"Found {len(results['masks'])} objects")